# [Sequential]

## _1_. 모듈 불러오기, 데이터 확인

In [ ]:
# module import
import numpy as np
import sys
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop
import random

In [ ]:
# 경로 설정
root_path = "/content/drive/My Drive/멀티캠퍼스/[혁신성장] 인공지능 자연어처리 기반/[강의]/조성현 강사님"
data_path = f"{root_path}/dataset"

In [ ]:
# 데이터 읽어 오기
with open(f"{data_path}/shakespeare_final.txt") as f:
    text = f.read().lower()

In [ ]:
print(text) # 공백도 있고, 특수문자도 있고, 개행문자도 있고, 등등.

the project gutenberg ebook of the complete works of william shakespeare, by
william shakespeare

this ebook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  you may copy it, give it away or
re-use it under the terms of the project gutenberg license included
with this ebook or online at www.gutenberg.org

** this is a copyrighted project gutenberg ebook, details below **
**     please follow the copyright guidelines in this file.     **

title: the complete works of william shakespeare

author: william shakespeare

posting date: september 1, 2011 [ebook #100]
release date: january, 1994

language: english


*** start of this project gutenberg ebook complete works--william shakespeare ***




produced by world library, inc., from their library of the future




this is the 100th etext file presented by project gutenberg, and
is presented in cooperation with world library, inc., from their
library of the future and shakespeare cdroms.  project gut

In [ ]:
# 데이터 확인
print(f"데이터 길이: {len(text)}")
characters = sorted(list(set(text))) # 영문자, 숫자, 특수문자 등 사용됨. 정렬.
print(f"총 문자 개수: {len(characters)}")

데이터 길이: 581432
총 문자 개수: 61


In [ ]:
# 인덱스, 문자 매핑 
char2idx = dict((c, i) for i, c in enumerate(characters))
idx2char = dict((i, c) for i, c in enumerate(characters))

In [ ]:
print(char2idx) # 문자 순서대로 매핑
print(idx2char) # 인덱스 - 문자, 문자 순서대로 매핑

{'\n': 0, ' ': 1, '!': 2, '"': 3, '#': 4, '&': 5, "'": 6, '(': 7, ')': 8, '*': 9, ',': 10, '-': 11, '.': 12, '/': 13, '0': 14, '1': 15, '2': 16, '3': 17, '4': 18, '5': 19, '6': 20, '7': 21, '8': 22, '9': 23, ':': 24, ';': 25, '<': 26, '>': 27, '?': 28, '@': 29, '[': 30, ']': 31, '_': 32, 'a': 33, 'b': 34, 'c': 35, 'd': 36, 'e': 37, 'f': 38, 'g': 39, 'h': 40, 'i': 41, 'j': 42, 'k': 43, 'l': 44, 'm': 45, 'n': 46, 'o': 47, 'p': 48, 'q': 49, 'r': 50, 's': 51, 't': 52, 'u': 53, 'v': 54, 'w': 55, 'x': 56, 'y': 57, 'z': 58, '|': 59, '~': 60}
{0: '\n', 1: ' ', 2: '!', 3: '"', 4: '#', 5: '&', 6: "'", 7: '(', 8: ')', 9: '*', 10: ',', 11: '-', 12: '.', 13: '/', 14: '0', 15: '1', 16: '2', 17: '3', 18: '4', 19: '5', 20: '6', 21: '7', 22: '8', 23: '9', 24: ':', 25: ';', 26: '<', 27: '>', 28: '?', 29: '@', 30: '[', 31: ']', 32: '_', 33: 'a', 34: 'b', 35: 'c', 36: 'd', 37: 'e', 38: 'f', 39: 'g', 40: 'h', 41: 'i', 42: 'j', 43: 'k', 44: 'l', 45: 'm', 46: 'n', 47: 'o', 48: 'p', 49: 'q', 50: 'r', 51: 's',

In [ ]:
# 텍스트 분절
max_length = int(input('최대 길이 설정: '))
n_step = int(input('시퀀스 길이 설정: ')) ## 3칸씩 뛰어 넘어서 sentence 생성
sentences = [] # 앞에 나오는 문장
next_chars = [] # 다음에 나올 문자
for i in range(0, len(text) - max_length, n_step): ## 
    sentences.append(text[i:i+max_length])
    next_chars.append(text[i+max_length])
print(f"문장 개수: {len(sentences)}")

최대 길이 설정: 40
시퀀스 길이 설정: 3
문장 개수: 193798


In [ ]:
for i in range(10):
    print(sentences[i], next_chars[i])

the project gutenberg ebook of the compl e
 project gutenberg ebook of the complete  
oject gutenberg ebook of the complete wo r
ct gutenberg ebook of the complete works  
gutenberg ebook of the complete works of  
enberg ebook of the complete works of wi l
erg ebook of the complete works of willi a
 ebook of the complete works of william  s
ook of the complete works of william sha k
 of the complete works of william shakes p


## _2_. 문장 벡터화

- 원핫 벡터화해서 하나의 array로 저장한다.


원핫 벡터화 과정 특이하다!

In [ ]:
# 원핫 벡터화: 3차원. 해당 위치에 나오는 character
X = np.zeros((len(sentences), max_length, len(characters)), dtype=np.bool) # 전부 다 False
y = np.zeros((len(sentences), len(characters)), dtype=np.bool)

In [ ]:
X.shape

(193798, 40, 61)

In [ ]:
y.shape

(193798, 61)

In [ ]:
for i, sent in enumerate(sentences):
    for t, char in enumerate(sent):
        X[i, t, char2idx[char]] = 1 # i번째 문장의 t 번째에 있는 문장 char = 문자의 위치 char2idx에서 찾아서 해당 위치를 1로 만듦.
    y[i, char2idx[next_chars[i]]] = 1 # i번째 문장 다음에 나와야 할 y를 char = 문자의 위치 char2idx에서 찾아서 해당 위치를 1로 만듦.

모델링
- 시퀀스 길이(D1축, 행) = max_length
- 피쳐 수(D2축, 열) = characters
- 아웃풋 노드 수 : characters 수. 무슨 문자 나올지.


In [ ]:
# 모델 파라미터 설정
n_hidden = int(input('은닉 노드 수 설정: '))

은닉 노드 수 설정: 128


In [ ]:
# 모델 네트워크
model = Sequential()
model.add(LSTM(n_hidden, input_shape=(max_length, len(characters))))
model.add(Dense(len(characters))) # 출력 노드 수
model.add(Activation('softmax')) # 출력층 활성화
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(learning_rate=0.01))
print("============ 모델 전체 구조 확인 ============")
print(model.summary())

============ 모델 전체 구조 확인 ============
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               97280     
_________________________________________________________________
dense (Dense)                (None, 61)                7869      
_________________________________________________________________
activation (Activation)      (None, 61)                0         
Total params: 105,149
Trainable params: 105,149
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
# 예측된 결과값을 인덱스로 바꾼다
def pred_indices(preds, metric=1.0):

    # exponential에 beta 취하는 부분
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / metric
    exp_preds = np.exp(preds)
    preds = exp_preds/ np.sum(exp_preds)

    probs = np.random.multinomial(1, preds, 1) # 다항분포에서 추출: 1번, pval=예측된 확률분포, 1개 뽑아라.
    
    return np.argmax(probs)

## _3_. 학습 및 예측
- 한 번 학습하고 한 번 generate하고 이게 반복인가?

In [ ]:
# 모델 훈련 및 평가
ITERS = int(input('반복 횟수 설정: '))
BATCH = int(input('배치 사이즈 설정: '))

for iter in range(1, ITERS):
    print('-'*40)
    print(f"ITERATION: {iter}")
    model.fit(X, y, batch_size=BATCH, epochs = 1) # 1번 학습한다.
    
    start_idx = random.randint(0, len(text)-max_length-1) # 아무 문장이나 찍어 올 인덱스

    for diversity in [0.2, 0.7, 1.2]: # softmax beta값을 각각 0.2, 0.7, 1.2로 바꿔 준다.
        print()
        print(f"       diversity: {diversity}")

        generated = ''
        sent = text[start_idx : start_idx+max_length] # 해당하는 문장을 문자 길이만큼 해서 가져 오고
        generated += sent # generated: 생성할 문장 처음에 생성할 초기값은 없으니까 sent.
        print(f"       ===== 초기 문장: '{sent}' ")
        sys.stdout.write(generated) # 콘솔에 초기 문장 출력

        for i in range(400):
            x = np.zeros((1, max_length, len(characters))) # 그 뒤에 나올 문장 예측할 벡터

            for t, char in enumerate(sent):
                x[0, t, char2idx[char]] = 1.0 # 문장 1개니까 면 축은 0, t번째에 나오는 char : char 위치 찾아서 1로 만든다.
            
            preds = model.predict(x, verbose=0)[0] # x를 넣고 그 다음에 나올 y를 찾는다. 2차원 배열이니까 인덱싱 통해 0번째로 잡아 준다.
            next_idx = pred_indices(preds, diversity) # 베타 값을 바꿔 주고 인덱스 찾도록
            pred_char = idx2char[next_idx] # 찾은 인덱스를 이용해 문자로 바꾸고

            generated += pred_char # 원래 문장 다음에 예측된 문자 붙인다.
            sent = sent[1:] + pred_char # 예측할 문장 업데이트 = sine곡선에서 array 길이 늘려주면서 예측하는 것과 동일.

            sys.stdout.write(pred_char)
            sys.stdout.flush()
        
        print()
        print("          한 조합 끝          ")

반복 횟수 설정: 30
배치 사이즈 설정: 128
----------------------------------------
ITERATION: 1
1515/1515 [==============================] - 7s 5ms/step - loss: 1.8934

       diversity: 0.2
       ===== 초기 문장: '               [a march afar]
  widow. a' 
               [a march afar]
  widow. a man him have have that heart that heart hame
    that hath that have have have have that hear that have have that have heart is thee
    as as as that have was hear have that heart that have that hath him as as that have that heart
    i shall have him as thee have that heart that hear hamp
    that hear head hame have have that heart in thee.
    and that hath that hear heart shall heart in the 
          한 조합 끝          

       diversity: 0.7
       ===== 초기 문장: '               [a march afar]
  widow. a' 
               [a march afar]
  widow. a most that cees. i lord of all mine.
    alone momse of mine as thy nothiod.
  helena. that stance sad in partion to mare use.
                         axe all that

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  


he reason she thou do sweet for your
    of elsemest halls thou do flies that thus
    the spring the great fortune.                                           exeunt's taughter- the ebroockngers with the complete wo
          한 조합 끝          

       diversity: 1.2
       ===== 초기 문장: ' whose age and honour
    both suffer un' 
 whose age and honour
    both suffer untoft withouts, hil. be goods
    the kings thrin's lie kindowar's recompellege
    of thou mura! if i such a doging; upod the full te.
  at her.
  an i can i know as might lo seens much jaqcipen,
  nature is my son i, to not to be auge
    therefore by them uncri'e, most reasons
    my matter, parding oathsawivery other-
    o, heaven, northing before youthing in me
  de aftor him shall his engros
          한 조합 끝          
----------------------------------------
ITERATION: 27
1515/1515 [==============================] - 7s 5ms/step - loss: 1.2383

       diversity: 0.2
       ===== 초기 문장: ' forsake ourselves. give me tha

# [강사님 교재 예시 코드]

> 결과 확인용

In [ ]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, LSTM, Activation
# from tensorflow.keras.optimizers import RMSprop
# import numpy as np
# import random
# import sys

# path = './dataset/shakespeare_final.txt'
# text = open(path).read().lower()

# 경로 설정
root_path = "/content/drive/My Drive/멀티캠퍼스/[혁신성장] 인공지능 자연어처리 기반/[강의]/조성현 강사님"
data_path = f"{root_path}/dataset"

# 데이터 읽어 오기
with open(f"{data_path}/shakespeare_final.txt") as f:
    text = f.read().lower()

print('corpus length:', len(text))
characters = sorted(list(set(text)))
print('total chars:', len(characters))

char2indices = dict((c, i) for i, c in enumerate(characters))
indices2char = dict((i, c) for i, c in enumerate(characters))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

# Converting indices into vectorized format
X = np.zeros((len(sentences), maxlen, len(characters)), dtype=np.bool)
y = np.zeros((len(sentences), len(characters)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char2indices[char]] = 1
    y[i, char2indices[next_chars[i]]] = 1
	
#Model Building
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(characters))))
model.add(Dense(len(characters)))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.01))
print (model.summary())

# Function to convert prediction into index
def pred_indices(preds, metric=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / metric
    exp_preds = np.exp(preds)
    preds = exp_preds/np.sum(exp_preds)
    probs = np.random.multinomial(1, preds, 1)
    return np.argmax(probs)

# Train & Evaluate the Model
for iteration in range(1, 30):
    print('-' * 40)
    print('Iteration', iteration)
    model.fit(X, y,batch_size=128,epochs=1)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.7,1.2]:
        print('\n----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x = np.zeros((1, maxlen, len(characters)))
            for t, char in enumerate(sentence):
                x[0, t, char2indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = pred_indices(preds, diversity)
            pred_char = indices2char[next_index]

            generated += pred_char
            sentence = sentence[1:] + pred_char

            sys.stdout.write(pred_char)
            sys.stdout.flush()
        print("\nOne combination completed \n")


corpus length: 581432
total chars: 61
nb sequences: 193798
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               97280     
_________________________________________________________________
dense (Dense)                (None, 61)                7869      
_________________________________________________________________
activation (Activation)      (None, 61)                0         
Total params: 105,149
Trainable params: 105,149
Non-trainable params: 0
_________________________________________________________________
None
----------------------------------------
Iteration 1
1515/1515 [==============================] - 12s 8ms/step - loss: 1.8849

----- diversity: 0.2
----- Generating with seed: "t him
    that he is lov'd of me; i foll"
t him
    that he is lov'd of me; i foll the sence shall we have son the prosess, and some the co

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:54: RuntimeWarning: divide by zero encountered in log


but the name me as so,
One combination completed 


----- diversity: 1.2
----- Generating with seed: "ion to try him.
  first lord. none bette"
ion to try him.
  first lord. none better of sudaming.
  helenas. madamking bhost. hi[d, sir.
  thatly thoug. by much chold-dimed
  erods, i like him in a knowcgene. he hassen.
  that is oft.
  what entery i abay.
  letir. thound of him, speanly darus!
  if i sored.
  rostorini. i brink up of.
  cloind. and icproute her forgottedc.
  hose-diana. minuth, no, o, gove charse
    of thatless. but, must ving's peace, and desire no leave acth
One combination completed 

----------------------------------------
Iteration 22
1515/1515 [==============================] - 12s 8ms/step - loss: 1.2569

----- diversity: 0.2
----- Generating with seed: "beguil'd.
  second guard. there's dolabe"
beguil'd.
  second guard. there's dolabella. she has the master of the world the master of the master of the world seem but the matter shalt
    the master the master 